<a href="https://www.kaggle.com/code/ryancardwell/seawolfprowlert?scriptVersionId=272636446" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 🏆 ARC Prize 2025 - Production Solver

**Strategy**: 13+ proven transformation patterns  
**Target**: 20-40% accuracy on hidden test set  
**Approach**: Pattern-based few-shot learning  

This solver implements real transformations that work on actual ARC tasks.

In [1]:
import json
import numpy as np
from typing import List, Dict, Tuple, Optional, Set
from collections import Counter, defaultdict
import copy

print("✅ Imports complete")

✅ Imports complete


## Grid Operations
Core primitives for grid manipulation

In [2]:
Grid = List[List[int]]

class GridOps:
    """Core grid manipulation primitives"""
    
    @staticmethod
    def to_numpy(grid: Grid) -> np.ndarray:
        return np.array(grid, dtype=np.int32)
    
    @staticmethod
    def from_numpy(arr: np.ndarray) -> Grid:
        return arr.tolist()
    
    @staticmethod
    def shape(grid: Grid) -> Tuple[int, int]:
        if not grid or not grid[0]:
            return (0, 0)
        return (len(grid), len(grid[0]))
    
    @staticmethod
    def grids_equal(g1: Grid, g2: Grid) -> bool:
        if not g1 or not g2:
            return g1 == g2
        h1, w1 = GridOps.shape(g1)
        h2, w2 = GridOps.shape(g2)
        if h1 != h2 or w1 != w2:
            return False
        return all(r1 == r2 for r1, r2 in zip(g1, g2))
    
    @staticmethod
    def rotate_90(grid: Grid, k: int = 1) -> Grid:
        arr = GridOps.to_numpy(grid)
        return GridOps.from_numpy(np.rot90(arr, -k))
    
    @staticmethod
    def flip_h(grid: Grid) -> Grid:
        return [row[::-1] for row in grid]
    
    @staticmethod
    def flip_v(grid: Grid) -> Grid:
        return grid[::-1]
    
    @staticmethod
    def transpose(grid: Grid) -> Grid:
        if not grid or not grid[0]:
            return grid
        return list(map(list, zip(*grid)))
    
    @staticmethod
    def get_colors(grid: Grid) -> Set[int]:
        return {cell for row in grid for cell in row}
    
    @staticmethod
    def get_background_color(grid: Grid) -> int:
        flat = [cell for row in grid for cell in row]
        if not flat:
            return 0
        return Counter(flat).most_common(1)[0][0]
    
    @staticmethod
    def replace_color(grid: Grid, old: int, new: int) -> Grid:
        return [[new if cell == old else cell for cell in row] for row in grid]
    
    @staticmethod
    def swap_colors(grid: Grid, c1: int, c2: int) -> Grid:
        return [[c2 if cell == c1 else c1 if cell == c2 else cell 
                 for cell in row] for row in grid]
    
    @staticmethod
    def tile(grid: Grid, rows: int, cols: int) -> Grid:
        arr = GridOps.to_numpy(grid)
        return GridOps.from_numpy(np.tile(arr, (rows, cols)))
    
    @staticmethod
    def scale_up(grid: Grid, factor: int) -> Grid:
        if factor <= 0:
            return grid
        result = []
        for row in grid:
            for _ in range(factor):
                scaled_row = []
                for cell in row:
                    scaled_row.extend([cell] * factor)
                result.append(scaled_row)
        return result
    
    @staticmethod
    def empty_grid(height: int, width: int, fill: int = 0) -> Grid:
        return [[fill] * width for _ in range(height)]

print("✅ GridOps defined")

✅ GridOps defined


## Pattern Detection
Analyze relationships between input/output grids

In [3]:
class PatternDetector:
    @staticmethod
    def detect_color_mapping(train_examples: List[Tuple[Grid, Grid]]) -> Optional[Dict[int, int]]:
        """Detect consistent color mappings across all examples"""
        if not train_examples:
            return None
        
        mapping = {}
        inp, out = train_examples[0]
        
        for i, row_in in enumerate(inp):
            for j, color_in in enumerate(row_in):
                if i < len(out) and j < len(out[0]):
                    color_out = out[i][j]
                    if color_in in mapping:
                        if mapping[color_in] != color_out:
                            return None
                    else:
                        mapping[color_in] = color_out
        
        for inp, out in train_examples[1:]:
            for i, row_in in enumerate(inp):
                for j, color_in in enumerate(row_in):
                    if i < len(out) and j < len(out[0]):
                        expected = mapping.get(color_in, color_in)
                        actual = out[i][j]
                        if expected != actual:
                            return None
        
        if all(k == v for k, v in mapping.items()):
            return None
        
        return mapping

print("✅ PatternDetector defined")

✅ PatternDetector defined


## Transformation Strategies
Each strategy checks if it applies to training examples, then applies to test

In [4]:
class Strategy:
    def __init__(self, name: str):
        self.name = name
    
    def can_apply(self, train_examples: List[Tuple[Grid, Grid]]) -> bool:
        return False
    
    def apply(self, test_input: Grid) -> Optional[Grid]:
        return None

class IdentityStrategy(Strategy):
    def __init__(self):
        super().__init__("identity")
    
    def can_apply(self, train_examples: List[Tuple[Grid, Grid]]) -> bool:
        return all(GridOps.grids_equal(inp, out) for inp, out in train_examples)
    
    def apply(self, test_input: Grid) -> Grid:
        return copy.deepcopy(test_input)

class Rotate90Strategy(Strategy):
    def __init__(self, k: int):
        super().__init__(f"rotate_{k*90}")
        self.k = k
    
    def can_apply(self, train_examples: List[Tuple[Grid, Grid]]) -> bool:
        return all(GridOps.grids_equal(GridOps.rotate_90(inp, self.k), out)
                   for inp, out in train_examples)
    
    def apply(self, test_input: Grid) -> Grid:
        return GridOps.rotate_90(test_input, self.k)

class FlipStrategy(Strategy):
    def __init__(self, direction: str):
        super().__init__(f"flip_{direction}")
        self.direction = direction
    
    def can_apply(self, train_examples: List[Tuple[Grid, Grid]]) -> bool:
        flip_fn = GridOps.flip_h if self.direction == 'h' else GridOps.flip_v
        return all(GridOps.grids_equal(flip_fn(inp), out) for inp, out in train_examples)
    
    def apply(self, test_input: Grid) -> Grid:
        return GridOps.flip_h(test_input) if self.direction == 'h' else GridOps.flip_v(test_input)

class TransposeStrategy(Strategy):
    def __init__(self):
        super().__init__("transpose")
    
    def can_apply(self, train_examples: List[Tuple[Grid, Grid]]) -> bool:
        return all(GridOps.grids_equal(GridOps.transpose(inp), out)
                   for inp, out in train_examples)
    
    def apply(self, test_input: Grid) -> Grid:
        return GridOps.transpose(test_input)

class ColorMappingStrategy(Strategy):
    def __init__(self):
        super().__init__("color_mapping")
        self.mapping = None
    
    def can_apply(self, train_examples: List[Tuple[Grid, Grid]]) -> bool:
        self.mapping = PatternDetector.detect_color_mapping(train_examples)
        if not self.mapping:
            return False
        for inp, out in train_examples:
            predicted = [[self.mapping.get(cell, cell) for cell in row] for row in inp]
            if not GridOps.grids_equal(predicted, out):
                return False
        return True
    
    def apply(self, test_input: Grid) -> Grid:
        if not self.mapping:
            return test_input
        return [[self.mapping.get(cell, cell) for cell in row] for row in test_input]

class ScaleUpStrategy(Strategy):
    def __init__(self):
        super().__init__("scale_up")
        self.factor = None
    
    def can_apply(self, train_examples: List[Tuple[Grid, Grid]]) -> bool:
        if not train_examples:
            return False
        inp, out = train_examples[0]
        h_in, w_in = GridOps.shape(inp)
        h_out, w_out = GridOps.shape(out)
        if h_in == 0 or w_in == 0:
            return False
        if h_out % h_in != 0 or w_out % w_in != 0:
            return False
        factor_h = h_out // h_in
        factor_w = w_out // w_in
        if factor_h != factor_w or factor_h == 1:
            return False
        self.factor = factor_h
        for inp, out in train_examples:
            scaled = GridOps.scale_up(inp, self.factor)
            if not GridOps.grids_equal(scaled, out):
                return False
        return True
    
    def apply(self, test_input: Grid) -> Grid:
        if not self.factor:
            return test_input
        return GridOps.scale_up(test_input, self.factor)

class TileStrategy(Strategy):
    def __init__(self):
        super().__init__("tile")
        self.rows = None
        self.cols = None
    
    def can_apply(self, train_examples: List[Tuple[Grid, Grid]]) -> bool:
        if not train_examples:
            return False
        inp, out = train_examples[0]
        h_in, w_in = GridOps.shape(inp)
        h_out, w_out = GridOps.shape(out)
        if h_in == 0 or w_in == 0:
            return False
        if h_out % h_in != 0 or w_out % w_in != 0:
            return False
        self.rows = h_out // h_in
        self.cols = w_out // w_in
        if self.rows == 1 and self.cols == 1:
            return False
        for inp, out in train_examples:
            tiled = GridOps.tile(inp, self.rows, self.cols)
            if not GridOps.grids_equal(tiled, out):
                return False
        return True
    
    def apply(self, test_input: Grid) -> Grid:
        if not self.rows or not self.cols:
            return test_input
        return GridOps.tile(test_input, self.rows, self.cols)

class GravityStrategy(Strategy):
    def __init__(self):
        super().__init__("gravity")
        self.background = 0
    
    def can_apply(self, train_examples: List[Tuple[Grid, Grid]]) -> bool:
        if not train_examples:
            return False
        self.background = GridOps.get_background_color(train_examples[0][0])
        for inp, out in train_examples:
            predicted = self._apply_gravity(inp)
            if not GridOps.grids_equal(predicted, out):
                return False
        return True
    
    def _apply_gravity(self, grid: Grid) -> Grid:
        h, w = GridOps.shape(grid)
        result = GridOps.empty_grid(h, w, self.background)
        for col in range(w):
            cells = []
            for row in range(h):
                if grid[row][col] != self.background:
                    cells.append(grid[row][col])
            start_row = h - len(cells)
            for i, cell in enumerate(cells):
                result[start_row + i][col] = cell
        return result
    
    def apply(self, test_input: Grid) -> Grid:
        return self._apply_gravity(test_input)

class MirrorStrategy(Strategy):
    def __init__(self, direction: str):
        super().__init__(f"mirror_{direction}")
        self.direction = direction
    
    def can_apply(self, train_examples: List[Tuple[Grid, Grid]]) -> bool:
        for inp, out in train_examples:
            predicted = self._create_mirror(inp)
            if not GridOps.grids_equal(predicted, out):
                return False
        return True
    
    def _create_mirror(self, grid: Grid) -> Grid:
        if self.direction == 'h':
            flipped = GridOps.flip_h(grid)
            return [row1 + row2 for row1, row2 in zip(grid, flipped)]
        else:
            flipped = GridOps.flip_v(grid)
            return grid + flipped
    
    def apply(self, test_input: Grid) -> Grid:
        return self._create_mirror(test_input)

print("✅ Strategies defined")

✅ Strategies defined


## Main Solver
Orchestrates strategy selection and application

In [5]:
class ARCSolver:
    def __init__(self):
        self.strategies = [
            IdentityStrategy(),
            Rotate90Strategy(1),
            Rotate90Strategy(2),
            Rotate90Strategy(3),
            FlipStrategy('h'),
            FlipStrategy('v'),
            TransposeStrategy(),
            ColorMappingStrategy(),
            ScaleUpStrategy(),
            TileStrategy(),
            GravityStrategy(),
            MirrorStrategy('h'),
            MirrorStrategy('v'),
        ]
        self.stats = {
            'total': 0,
            'solved_by_strategy': defaultdict(int),
            'fallback': 0
        }
    
    def solve_task(self, train_examples: List[Tuple[Grid, Grid]], 
                   test_inputs: List[Grid]) -> List[List[Grid]]:
        results = []
        for test_input in test_inputs:
            best_solution = None
            best_strategy_name = "fallback"
            
            for strategy in self.strategies:
                try:
                    if strategy.can_apply(train_examples):
                        solution = strategy.apply(test_input)
                        if solution is not None:
                            best_solution = solution
                            best_strategy_name = strategy.name
                            break
                except Exception:
                    continue
            
            if best_solution is None:
                best_solution = copy.deepcopy(test_input)
                self.stats['fallback'] += 1
            else:
                self.stats['solved_by_strategy'][best_strategy_name] += 1
            
            self.stats['total'] += 1
            results.append([best_solution, best_solution])
        
        return results

print("✅ ARCSolver defined")

✅ ARCSolver defined


## Main Execution
Load data, solve tasks, generate submission

In [6]:
print("="*80)
print("ARC PRIZE 2025 - PRODUCTION SOLVER")
print("="*80)

# Paths
CHALLENGES_PATH = '/kaggle/input/arc-prize-2025/arc-agi_test_challenges.json'
OUTPUT_PATH = '/kaggle/working/submission.json'

# Load challenges
print(f"\n📚 Loading challenges...")
with open(CHALLENGES_PATH, 'r') as f:
    challenges = json.load(f)
print(f"   Found {len(challenges)} tasks")

# Initialize solver
solver = ARCSolver()
print(f"\n🔧 Initialized {len(solver.strategies)} strategies")

# Solve all tasks
print("\n🚀 Solving tasks...")
submission = {}

for i, (task_id, task_data) in enumerate(challenges.items()):
    if (i + 1) % 20 == 0:
        print(f"   Progress: {i+1}/{len(challenges)}")
    
    train_examples = [(ex['input'], ex['output']) for ex in task_data['train']]
    test_inputs = [ex['input'] for ex in task_data['test']]
    
    predictions = solver.solve_task(train_examples, test_inputs)
    submission[task_id] = predictions

print(f"   Complete: {len(challenges)}/{len(challenges)}")

# Save submission
print(f"\n💾 Saving submission...")
with open(OUTPUT_PATH, 'w') as f:
    json.dump(submission, f)

# Print statistics
print("\n" + "="*80)
print("STATISTICS")
print("="*80)
print(f"Total test inputs: {solver.stats['total']}")
print(f"Fallback (input copied): {solver.stats['fallback']}")
print(f"\nStrategies used:")
for strategy, count in sorted(solver.stats['solved_by_strategy'].items(), 
                             key=lambda x: x[1], reverse=True):
    print(f"  {strategy}: {count}")

print("\n" + "="*80)
print("✅ COMPLETE!")
print(f"   File: {OUTPUT_PATH}")
print("="*80)

ARC PRIZE 2025 - PRODUCTION SOLVER

📚 Loading challenges...
   Found 240 tasks

🔧 Initialized 13 strategies

🚀 Solving tasks...
   Progress: 20/240
   Progress: 40/240
   Progress: 60/240
   Progress: 80/240
   Progress: 100/240
   Progress: 120/240
   Progress: 140/240
   Progress: 160/240
   Progress: 180/240
   Progress: 200/240
   Progress: 220/240
   Progress: 240/240
   Complete: 240/240

💾 Saving submission...

STATISTICS
Total test inputs: 259
Fallback (input copied): 257

Strategies used:
  gravity: 1
  rotate_180: 1

✅ COMPLETE!
   File: /kaggle/working/submission.json
